## Summarizing the text before applying LDA

In [1]:
# !pip install --user pytextrank
# !pip install --user ktrain

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import sys
import os
import nltk.corpus  
from nltk.text import Text
import spacy
from pandarallel import pandarallel
import multiprocessing
import warnings
warnings.filterwarnings('ignore')
import ipywidgets as widgets
widgets.IntSlider()
pd.set_option('max_colwidth', 200)
import pytextrank

2023-05-23 15:46:03.857788: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-23 15:46:08.490162: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-05-23 15:46:08.491179: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [15]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("textrank")
nlp.add_pipe("sentencizer")

In [16]:
print(nlp.pipe_names)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner', 'textrank', 'sentencizer']


In [4]:
%%time
df = pd.read_parquet('data_to_sentiment.parquet', engine='pyarrow')
df.shape

CPU times: user 32 s, sys: 8.95 s, total: 41 s
Wall time: 51.9 s


(181958, 14)

In [9]:
text = df['text_clean'][0]
text

'Artificial intelligence improves parking efficiency Chinese cities People Daily Online Home China Politics Foreign Affairs Opinions Video We Are China Business Military World Society Culture Travel Science Sports Photo Languages Chinese Japanese French Spanish Russian Arabic Korean German Portuguese Thursday , March 18 , 2021 Home Artificial intelligence improves parking efficiency Chinese cities By LiuShiyao People Daily 09 16 , March 18 , 2021 Photo taken July 1 , 2019 , shows sign electronic toll collection ETC newly set roadside parking space Yangzhuang road , Shijingshan district , Beijing . Some urban areas city started use ETC system roadside parking spaces July 1 , 2019 . Peoples Daily Online Li Wenming Thanks application artificial intelligence AI empowered roadside electronic toll collection ETC system , Chinas city Beijing seen significant improvement efficiency parking fee collection , turnover roadside parking spots , order roadside parking , congestion . As city deepens 

In [42]:
doc = nlp(text)
sumy = (list(doc._.textrank.summary(limit_phrases = 20, limit_sentences=15)))
summaries = [span.text for span in sumy]
' '.join(summaries)

'Peoples Daily Online Li Wenming Thanks application artificial intelligence AI empowered roadside electronic toll collection ETC system , Chinas city Beijing seen significant improvement efficiency parking fee collection , turnover roadside parking spots , order roadside parking , congestion . As city deepens roadside parking reform , ETC system covered roadside parking spaces city , proportion vehicles parked roads system exceeding 90 percent . The companys intelligent system brought play advantages AI technologies effectively addressed shortage parking spaces problem irregular parking cities . So far , smart ETC system AIpark introduced 20 cities China , signaling increasingly important roles AI improving parking efficiency order new development opportunities smart parking industry . There deficiencies traditional parking systems . Some urban areas city started use ETC system roadside parking spaces July 1 , 2019 . Artificial intelligence improves parking efficiency Chinese cities We

In [46]:
def process_text(text):
    doc = nlp(text)
    sumy_obj = list(doc._.textrank.summary(limit_phrases =20, limit_sentences = 15))
    summary = [span.text for span in sumy_obj]
    return summary

In [44]:
pandarallel.initialize(nb_workers = 62, progress_bar = True, use_memory_fs = False)

INFO: Pandarallel will run on 62 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [47]:
%%time
df['summary'] = df['text_clean'].parallel_apply(lambda x: process_text(x))

CPU times: user 3min 36s, sys: 44.6 s, total: 4min 20s
Wall time: 15min 40s


In [50]:
%%time
df['summary'] = df['summary'].parallel_apply(lambda x: " ".join(x))

CPU times: user 1.93 s, sys: 5.95 s, total: 7.88 s
Wall time: 10.8 s


In [51]:
df[['text_clean','summary']].head(3)

,text_clean,summary
0,Artificial intelligence improves parking efficiency Chinese cities People Daily Online Home China Politics Foreign Affairs Opinions Video We Are China Business Military World Society Culture Trave...,"Peoples Daily Online Li Wenming Thanks application artificial intelligence AI empowered roadside electronic toll collection ETC system , Chinas city Beijing seen significant improvement efficiency..."
1,"Children With Autism Saw Their Learning Social Skills Boosted After Playing With This AI Robot News Parliament Skip Thursday , February 27 , 2020 Latest Mansplaining conferences How forestall Drax...","Overnight Like Did Italy February 27 , 2020February 27 , 2020 admin News Coronavirus Dettol sales surge markets fall February 27 , 2020February 27 , 2020 Allen Becker Business Levi Strauss marks p..."
2,"Forget ML , AI Industry 4.0 obsolescence focus 26 February 2021 Test Rework Solutions Dataweek Home About Back issues book PDF EMP Handbook Subscribe Advertise Categories Editor Choice Multimedia ...","Forget ML , AI Industry 4.0 obsolescence focus 26 February 2021 Test Rework Solutions Dataweek Home About Back issues book PDF EMP Handbook Subscribe Advertise Categories Editor Choice Multimedia ..."


In [52]:
%%time
df.to_parquet("data_to_sentiment.parquet")

CPU times: user 36 s, sys: 7.52 s, total: 43.6 s
Wall time: 52.9 s


In [53]:
import datetime
import time
import pytz
datetime.datetime.now(pytz.timezone('US/Central')).strftime("%a, %d %B %Y %H:%M:%S")

'Tue, 23 May 2023 11:41:33'